In [35]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fremo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
df = pd.read_csv("spotify_millsongdata.csv")

In [37]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [38]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [39]:
df.shape

(57650, 4)

In [40]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [41]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [42]:
df.head(10)

,artist,song,text
0,Rush,How It Is,Here's a little trap \r\nThat sometimes catch...
1,Cinderella,Freewheelin',You say I get around but \r\nI'm gonna hit th...
2,John Mellencamp,Golden Gates,Ain't no golden gates gonna swing open \r\nAi...
3,Kanye West,Hold On (Remix),(Ohh Ohh ohh \r\nOohhh Oohhh Oohhh \r\nHold ...
4,Reba Mcentire,cry,"I might bite my lip, look down at my shoes \r..."
5,Blur,I Got Law,Feeling daft beyond gone bland \r\nMade it to...
6,Kate Bush,Cussi Cussi,Please \r\nDon't be-e afraid of this \r\nCoz...
7,Kim Wilde,Young Heroes,"Everyone round keeps putting us down, let it g..."
8,David Bowie,Can You Hear Me,"Once we were lovers, can they understand? \r\..."
9,Leann Rimes,You Made Me Find Myself,Bet you thought my world \r\nwas over \r\nBe...


In [43]:
df['text'][0]

"Here's a little trap  \r\nThat sometimes catches everyone  \r\nWhen today's as far as we can see  \r\nFaith in bright tomorrows  \r\nGiving way to resignation  \r\nThat's how it is - how it's going to be  \r\n  \r\nIt's such a cloudy day  \r\nSeems we'll never see the sun  \r\nOr feel the day has possibilities  \r\nFrozen in the moment -  \r\nThe lack of imagination  \r\nBetween how it is and how it ought to be  \r\n  \r\nHere's a little trap  \r\nThat sometimes trips up everyone  \r\nWhen we tire of our own company  \r\nSometimes we're the last to see beyond the day's frustrations  \r\nThat's how it is - how it's going to be  \r\n  \r\nIt's such a cloudy day  \r\nSeems we'll never see the sun  \r\nI feel the day is all uncertainty  \r\nBurning in the moment - trapped by the desperation  \r\nBetween how it is and how it ought to be  \r\n  \r\nFoot upon the stair  \r\nShoulder to the wheel  \r\nYou can't tell yourself not to care  \r\nYou can't tell yourself how to feel  \r\n  \r\nThat

In [44]:
# df = df.sample(5000)

In [45]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [46]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [47]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [48]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [51]:
similarity[0]

array([1.        , 0.00630316, 0.02570041, ..., 0.04141794, 0.00474073,
       0.00671054])

In [52]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3056,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [53]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [54]:
recommendation('Crying Over You')

['Cry Baby',
 'Cry For Help',
 'Cry, Cry, Cry',
 'Cry Baby',
 'No Woman',
 'True Love Always',
 'Cry Cry Cry',
 'Cry For Love',
 'Seven Lonely Days',
 'Cry Clock Said',
 'I Saw It In The Mirror',
 'Fool To Cry',
 "Big Girls Don't Cry",
 "Don't Cry Daddy",
 "It's The Little Things",
 'I Wish It Would Rain',
 'Sonny Cried',
 'House On The Ocean',
 "I'll Cry Instead",
 'Two Tears']

In [55]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))